importacion de librerias


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [3]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
33048,2024S026048,13/09/2024,16:23:00,"AVDA. VENTISQUERO DE LA CONDESA, 8",8,8,FUENCARRAL-EL PARDO,Alcance,Despejado,Turismo,Pasajero,De 40 a 44 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,439267.0,4482067.0,N,NaN
5191,2024S005050,09/02/2024,17:55:00,"BULEV. JOSE PRAT, 40",40,19,VICÁLVARO,Alcance,NaN,Turismo,Pasajero,De 40 a 44 años,Mujer,NaN,NaN,447347.0,4471986.0,N,NaN
19383,2024S017516,24/05/2024,9:10:00,CALL. JOSE DEL HIERRO / CALL. FEDERICO GUTIERREZ,5,15,CIUDAD LINEAL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,6.0,Asistencia sanitaria inmediata en centro de sa...,445104.0,4476539.0,N,NaN
5255,2024S005085,06/02/2024,0:00:00,"CALL. VENTURA RODRIGUEZ, 20",20,9,MONCLOA-ARAVACA,Choque contra obstáculo fijo,Lluvia débil,Turismo,Conductor,De 50 a 54 años,Hombre,NaN,NaN,439412.0,4475128.0,N,NaN
27064,2024S022324,17/07/2024,18:27:00,"CALL. LOS EMIGRANTES, 3",3,16,HORTALEZA,Colisión múltiple,Despejado,Turismo,Conductor,De 40 a 44 años,Mujer,14.0,Sin asistencia sanitaria,446044.0,4479695.0,N,NaN
7194,2024S006973,24/02/2024,16:50:00,CALL. CLARA CAMPOAMOR / CALL. CARCASTILLO,75,11,CARABANCHEL,Colisión fronto-lateral,Nublado,Turismo,Conductor,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,437801.0,4470113.0,N,NaN
19610,2024S017944,25/05/2024,13:40:00,"PASEO. PRADO, 34",34,1,CENTRO,Atropello a persona,Despejado,Turismo,Peatón,De 40 a 44 años,Mujer,2.0,Ingreso inferior o igual a 24 horas,441211.0,4473593.0,N,NaN
22680,2024S019726,13/06/2024,22:00:00,"AUTOV. M-30, CALZADA INT. LATERAL PK 7.700",07NL70,3,RETIRO,Choque contra obstáculo fijo,Despejado,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,444098.0,4474514.0,N,NaN
2964,2024S002658,24/01/2024,21:30:00,CALL. ALBERTO AGUILERA / CALL. VALLEHERMOSO,1,1,CENTRO,Colisión lateral,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,439856.0,4475726.0,N,NaN
16718,2024S015557,05/05/2024,21:25:00,CALL. CRISTOBAL BORDIU / CALL. MODESTO LAFUENTE,52,7,CHAMBERÍ,Caída,LLuvia intensa,Bicicleta,Conductor,De 21 a 24 años,Mujer,1.0,Atención en urgencias sin posterior ingreso,440973.0,4477147.0,N,NaN


Analisis del dataset


https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf


Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.


**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm


para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion


para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)


en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding


para el **rango de edad** vamos


Entendimiento del Dataset


Limpieza


Explicacion de las acciones realizadas


In [4]:
len(df['tipo_vehiculo'].unique())


33

In [5]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [6]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [7]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

Mapeo de rango de edades


In [8]:
map_edad = {
    'Desconocido': -1,
    'Menor de 5 años': 0,
    'De 6 a 9 años': 0,
    'De 10 a 14 años': 10,
    'De 15 a 17 años': 10,
    'De 18 a 20 años': 18,
    'De 21 a 24 años': 18,
    'De 25 a 29 años': 25,
    'De 30 a 34 años': 25,
    'De 35 a 39 años': 25,
    'De 40 a 44 años': 40,
    'De 45 a 49 años': 40,
    'De 50 a 54 años': 40,
    'De 55 a 59 años': 40,
    'De 60 a 64 años': 60,
    'De 65 a 69 años': 60,
    'De 70 a 74 años': 60,
    'Más de 74 años': 60
}

In [9]:
df['rango_edad'] = df['rango_edad'].map(map_edad)

In [19]:
valores_edad = df['rango_edad'].unique()


[40 25 60 18 -1 10  0]
40
25
60
18
-1
10
0


In [17]:
print(valores_edad.sort())

None


Explicación: Para simplificar y transformar esta columna, debido a que estaba definida con rangos, hemos decidido solo poner los valores con los que empieza el rango

Mapeo de estado meteorológico


In [10]:
df = df.fillna({
    'estado_meteorológico': 'Se desconoce'
})

print(df['estado_meteorológico'].unique())

map_est_met = {
    'Se Desconoce': -1,
    'Despejado': 0,
    'Nublado': 1,
    'Lluvia débil': 2,
    'LLuvia intensa': 3,
    'Granizando': 4,
    'Nevando': 5
}

df['estado_meteorológico'] = df['estado_meteorológico'].map(map_est_met)

['Lluvia débil' 'Despejado' 'Se desconoce' 'Nublado' 'LLuvia intensa'
 'Granizando' 'Nevando']


Mapeo Lesividad


In [ ]:
print(df['lesividad'].unique())

In [ ]:
df = df.fillna({
    'lesividad': 'Sin asistencia sanitaria',
    'cod_lesividad': 0
})

map_lesividad = {
    'Sin asistencia sanitaria': 'NONE',
    'Atención en urgencias sin posterior ingreso': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'LEVE',
    'Asistencia sanitaria ambulatoria con posterioridad': 'LEVE',
    'Asistencia sanitaria inmediata en centro de salud o mutua': 'LEVE',
    'Asistencia sanitaria sólo en el lugar del accidente': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'GRAVE',
    'Fallecido 24 horas': 'FALLECIDO'
}

In [ ]:
df['lesividad'] = df['lesividad'].map(map_lesividad)

In [ ]:
df['lesividad'].value_counts()

Mapeo Alcohol y drogas


In [ ]:
df = df.fillna({
    'positiva_droga': 0
})

map_alcohol = {
    'N': 0,
    'S': 1
}

df['positiva_alcohol'] = df['positiva_alcohol'].map(map_alcohol)

In [ ]:
print('valores de positivos en alcohol: \n {} \n valores de positivos en droga: \n {}'.format(
    df['positiva_alcohol'].value_counts(),df['positiva_droga'].value_counts()
    ))

In [ ]:
df.head(10)

In [ ]:
df['rango_edad'].value_counts()

In [ ]:
df.columns

In [ ]:
df['cod_lesividad'].unique()

In [ ]:
df['cod_lesividad'] = df['cod_lesividad'].astype(int)

In [ ]:
df['year'] = df['num_expediente'].astype(str).str[:4].astype(int)
df['id'] = df['num_expediente'].astype(str).str[-6:]

In [ ]:
df['year'].value_counts()

In [ ]:
df = pd.get_dummies(df, columns=['tipo_persona'])

In [ ]:
df.head(10)

In [ ]:
df.columns

In [ ]:
mapeo_accidentes = {
    'NONE':0,
    'LEVE':1,
    'GRAVE':2,
    'FALLECIDO':3
}

df['lesividad'] = df['lesividad'].map(mapeo_accidentes)

In [ ]:
map_tipo_accidente = {
    'Colisión fronto-lateral': 'Colision doble',
    'Alcance': 'Alcance',
    'Colisión lateral': 'Colision doble',
    'Choque contra obstáculo fijo':'Choque',
    'Atropello a persona': 'Atropello a persona',
    'Caída':'Caida',
    'Colisión múltiple':'Colision multiple',
    'Colisión frontal':'Colision doble',
    'Otro':'Otras Causas',
    'Solo salida de la vía': 'Otras Causas',
    'Vuelco': 'Vuelco',
    'Atropello a animal': 'Otras Causas'
}

df['tipo_accidente'] = df['tipo_accidente'].map(map_tipo_accidente)

In [ ]:
df['hora'] = df['hora'].str.split(':').str[0].astype(int)
df['hora'].value_counts()

In [ ]:
df_entrenamiento = df.drop(columns=['num_expediente', 'localizacion', 'numero', 'distrito', 'lesividad'], axis=1)

In [ ]:
df_entrenamiento.head(10)

Group by por id de accidente:

-   Hora a solo la hora
-   Rango de edad a sumatorio de edades
-   Tipo vehiculo one hot encoding y suma
-   Tipo de persona one hot encoding y suma
-   Sexo one hot encoding y suma
-   Positivo en drogas y alcohol sumar el numero de positivos
-   Lesividad la mas alta
